In [ ]:
pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import nltk
import nlpaug.augmenter.word as naw

# Download the WordNet resource from NLTK
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def augment_text_with_synonyms(text, num_augments=1):
    # Create an augmenter using the Synonym Augmenter, which by default uses WordNet
    augmenter = naw.SynonymAug(aug_src='wordnet', aug_max=num_augments)

    # Apply the augmentation to the input text
    augmented_texts = [augmenter.augment(text) for _ in range(num_augments)]

    return augmented_texts

# Example usage
original_text = "The quick brown fox jumps over the lazy dog."
augmented_texts = augment_text_with_synonyms(original_text, num_augments=3)

print("Original:", original_text)
for i, aug_text in enumerate(augmented_texts, 1):
    print(f"Augmented {i}:", aug_text)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Original: The quick brown fox jumps over the lazy dog.
Augmented 1: ['The warm brown charles james fox rise over the lazy dog.']
Augmented 2: ['The straightaway brown university fox jump over the lazy dog.']
Augmented 3: ['The spry brownness dodger jumps over the lazy dog.']


In [ ]:
import pandas as pd
Train = pd.read_csv('/content/task2_train_02_05_24.csv')
Val = pd.read_csv('/content/task2_dev_02_05_24.csv')

In [ ]:
Train = Train[['tweet', 'task2_hard_labels']]
Train

,tweet,task2_hard_labels
0,"@TheChiflis Ignora al otro, es un capullo.El p...",REPORTED
1,@ultimonomada_ Si comicsgate se parece en algo...,-
2,"@Steven2897 Lee sobre Gamergate, y como eso ha...",-
3,@Lunariita7 Un retraso social bastante lamenta...,-
4,@novadragon21 @icep4ck @TvDannyZ Entonces como...,UND
...,...,...
6915,idk why y’all bitches think having half your a...,DIRECT
6916,This has been a part of an experiment with @Wo...,JUDGEMENTAL
6917,"""Take me already"" ""Not yet. You gotta be ready...",DIRECT
6918,@clintneedcoffee why do you look like a whore?...,DIRECT


In [ ]:
Val = Val[['tweet', 'task2_hard_labels']]
Val

,tweet,task2_hard_labels
0,@Fichinescu La comunidad gamer es un antro de ...,-
1,@anacaotica88 @MordorLivin No me acuerdo de lo...,JUDGEMENTAL
2,@cosmicJunkBot lo digo cada pocos dias y lo re...,-
3,Also mientras les decia eso la señalaba y deci...,REPORTED
4,"And all people killed, attacked, harassed by ...",-
...,...,...
1033,@YesReallyAngel “Don’t wear a black bra with a...,DIRECT
1034,""" get changed , you look like a prostitute . ""...",REPORTED
1035,made this top and my mom gave me the “you look...,REPORTED
1036,@DawnAnd91320913 I haven't seen anything that ...,DIRECT


In [ ]:
Train['task2_hard_labels'].value_counts()

task2_hard_labels
-              3987
DIRECT         1275
UND             856
REPORTED        442
JUDGEMENTAL     360
Name: count, dtype: int64

In [ ]:
Train = Train[Train['task2_hard_labels'] != 'UND']

In [ ]:
Val = Val[Val['task2_hard_labels'] != 'UND']

In [ ]:
Train['task2_hard_labels'].value_counts()

task2_hard_labels
-              3987
DIRECT         1275
REPORTED        442
JUDGEMENTAL     360
Name: count, dtype: int64

In [ ]:
Val['task2_hard_labels'].value_counts()

task2_hard_labels
-              557
DIRECT         201
JUDGEMENTAL     79
REPORTED        73
Name: count, dtype: int64

In [ ]:
df = pd.concat([Train, Val], axis=0)
df

,tweet,task2_hard_labels
0,"@TheChiflis Ignora al otro, es un capullo.El p...",REPORTED
1,@ultimonomada_ Si comicsgate se parece en algo...,-
2,"@Steven2897 Lee sobre Gamergate, y como eso ha...",-
3,@Lunariita7 Un retraso social bastante lamenta...,-
5,@yonkykong Aaah sí. Andrew Dobson. El que se d...,-
...,...,...
1033,@YesReallyAngel “Don’t wear a black bra with a...,DIRECT
1034,""" get changed , you look like a prostitute . ""...",REPORTED
1035,made this top and my mom gave me the “you look...,REPORTED
1036,@DawnAnd91320913 I haven't seen anything that ...,DIRECT


In [ ]:
max_size = df['task2_hard_labels'].value_counts().max()

In [ ]:
def augment_text_with_synonyms(text, num_augments=1):
    augmenter = naw.SynonymAug(aug_src='wordnet', aug_max=num_augments)
    augmented_texts = [augmenter.augment(text) for _ in range(num_augments)]
    return augmented_texts

def balance_classes(df, text_column, label_column):
    # Calculate the maximum size among the classes
    class_counts = df[label_column].value_counts()
    max_size = class_counts.max()

    # Create a list to hold the augmented DataFrame pieces
    results = []

    # Loop through each class and augment if necessary
    for class_label, count in class_counts.items():
        class_subset = df[df[label_column] == class_label]
        results.append(class_subset)
        num_required = max_size - count  # Number of additional samples needed

        if num_required > 0:
            augmented_texts = []
            # Continue to augment the data until we reach the required number of samples
            while len(augmented_texts) < num_required:
                for text in class_subset[text_column]:
                    # Limit the number of augments to the number we still need
                    remaining = num_required - len(augmented_texts)
                    augments = augment_text_with_synonyms(text, min(1, remaining))
                    augmented_texts.extend(augments)
                    if len(augmented_texts) >= num_required:
                        break

            # Create a new DataFrame from the augmented texts and append it
            augmented_df = pd.DataFrame(augmented_texts, columns=[text_column])
            augmented_df[label_column] = class_label
            results.append(augmented_df)

    # Concatenate all the pieces of the DataFrame back together
    balanced_df = pd.concat(results).reset_index(drop=True)
    return balanced_df

In [ ]:
balanced_df = balance_classes(df, 'tweet', 'task2_hard_labels')

In [ ]:
balanced_df

,tweet,task2_hard_labels
0,@ultimonomada_ Si comicsgate se parece en algo...,-
1,"@Steven2897 Lee sobre Gamergate, y como eso ha...",-
2,@Lunariita7 Un retraso social bastante lamenta...,-
3,@yonkykong Aaah sí. Andrew Dobson. El que se d...,-
4,@glutamatom @JoaquinAdolfoC Estaba del lado de...,-
...,...,...
18171,CASADO MOLESTO CON SU SONRISA IRÓNICA POR QUÉ ...,JUDGEMENTAL
18172,"@ Miquel_R @ Jorge_Tamames Claro, que haya hij...",JUDGEMENTAL
18173,@ S_EkirouTarouX @ yanosoyootu @ menosloboseh ...,JUDGEMENTAL
18174,@ Asta_Demoon es una estructura estructural es...,JUDGEMENTAL


In [ ]:
balanced_df['task2_hard_labels'].value_counts()

task2_hard_labels
-              4544
DIRECT         4544
REPORTED       4544
JUDGEMENTAL    4544
Name: count, dtype: int64

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split



def split_data(df, label_column, train_size=0.85):
    # Shuffle the DataFrame
    df = df.sample(frac=1).reset_index(drop=True)

    # Split the data while maintaining equal distribution across classes
    train_df, val_df = train_test_split(df, stratify=df[label_column], train_size=train_size, random_state=42)

    return train_df, val_df

# Example usage with a DataFrame 'balanced_df' with equal class distribution
# Assuming 'balanced_df' has columns 'text' and 'label'
train_df, val_df = split_data(balanced_df, 'task2_hard_labels')

print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Training set class distribution:\n", train_df['task2_hard_labels'].value_counts())
print("Validation set class distribution:\n", val_df['task2_hard_labels'].value_counts())

Training set size: 15449
Validation set size: 2727
Training set class distribution:
 task2_hard_labels
DIRECT         3863
REPORTED       3862
JUDGEMENTAL    3862
-              3862
Name: count, dtype: int64
Validation set class distribution:
 task2_hard_labels
JUDGEMENTAL    682
REPORTED       682
-              682
DIRECT         681
Name: count, dtype: int64


In [ ]:
train_df.to_csv('balanced_train.csv', index=False)
val_df.to_csv('balanced_val.csv', index=False)